In [1]:
pip install newspaper3k

Note: you may need to restart the kernel to use updated packages.


In [2]:
import feedparser as fp
import json
import newspaper
from newspaper import Article
from time import mktime
from datetime import datetime

In [19]:
limit = 50

In [26]:
data = {}
data['newspapers'] = {}

In [27]:
data

{'newspapers': {}}

In [28]:
with open('data/NewsPapers.json') as data_file:
    companies = json.load(data_file)

In [33]:
count = 1
for company, value in companies.items():
    if 'rss' in value:
            d = fp.parse(value['rss'])
            print("Downloading articles from ", company)
            newsPaper = {
                "rss": value['rss'],
                "link": value['link'],
                "articles": []
            }


In [36]:
count = 1
for company, value in companies.items():
    if 'rss' in value:
            d = fp.parse(value['rss'])
            print("Downloading articles from ", company)
            newsPaper = {
                "rss": value['rss'],
                "link": value['link'],
                "articles": []
            }

            for entry in d.entries:
                if hasattr(entry,'published'):
                    if count > limit:
                        break
                    article = {}
                    article['link'] = entry.link
                    date = entry.published_parsed
                    article['published'] = datetime.fromtimestamp(mktime(date)).isoformat()
                    try:
                        content = Article(entry.link)
                        content.download()
                        content.parse()
                    except Exception as e:
                        print(e)
                        print("continuing...")
                        continue
                    article['title'] = content.title
                    article['text'] = content.text
                    article['author'] = content.authors
                    newsPaper['articles'].append(article)
                    if count % 10 == 0:
                        print(count, "articles downloaded from ", company, ", url: ", entry.link)
                    count = count + 1
    else:
        print("Buiding site for ", company)
        paper = newspaper.build(value['link'], memorize_articles = False)
        newsPaper = {
            "link": value['link'],
            "articles": []
        }
        noneTypeCount = 0
        for content in paper.articles:
            if count > limit:
                break
            try:
                content.download()
                content.parse()
            except Exception as e:
                print(e)
                print("continuing...")
                continue
            if content.publish_date is None:
                print(count, "Article has date of type None...")
                noneTypeCount = noneTypeCount + 1
                if noneTypeCount > 10:
                    print("Too many noneTypes dates, aborting...")
                    noneTypeCount = 0
                    break
                count = count + 1
                continue
            article = {}
            article['title'] = content.title
            article['text'] = content.text
            article['link'] = content.url
            article['published'] = content.publish_date.isoformat()
            article['author'] = content.authors
            newsPaper['articles'].append(article)
            if count % 10 == 0:
                 print(count, "articles downloaded from", company, " using newspaper, url: ", content.url)
            count = count + 1
            noneTypeCount = 0
    count = 1
    data['newspapers'][company] = newsPaper

try:
    fname = 'data/scraped_articles.json'
    print('saving articles . . . in {}'.format(fname))
    with open(fname, 'w') as outfile:
        json.dump(data, outfile)
except Exception as e: print(e)   
                

10 articles downloaded from  newyorktimes_business , url:  https://www.nytimes.com/2020/05/16/health/fda-clears-another-coronavirus-testing-kit-for-use-at-home.html
20 articles downloaded from  newyorktimes_business , url:  https://www.nytimes.com/2020/05/15/business/covid-mortgage-forbearance.html
30 articles downloaded from  newyorktimes_business , url:  https://www.nytimes.com/2020/05/15/your-money/coronavirus-credit-reports.html
40 articles downloaded from  newyorktimes_business , url:  https://www.nytimes.com/2020/05/15/business/movie-theaters-coronavirus.html
50 articles downloaded from  newyorktimes_business , url:  https://www.nytimes.com/2020/05/14/business/media/quartz-to-lay-off-80-employees.html
10 articles downloaded from  newyorktimes_science , url:  https://www.nytimes.com/2020/05/16/obituaries/peter-brancazio-dead-coronavirus.html
20 articles downloaded from  newyorktimes_science , url:  https://www.nytimes.com/2020/05/14/world/asia/coronavirus-testing-china-wuhan.html
